#### Provide a frequency distribution of the vital signs/physical exam readings observed across the dataset

In [8]:
import os
import glob
import xml.etree.ElementTree as ET
import re
import pandas as pd
from collections import Counter

In [2]:
## Finding all patterns of vital signs in the text

directory = 'Project3_data'
xml_files = glob.glob(os.path.join(directory, '*.xml'))
vital_signs_pattern = {}
count = 0
files_with_vital_signs = []
for xml_file in xml_files:
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for elem in root.iter('TEXT'):
        if elem.text:
            matches = re.findall(r'\b(vital signs|Vital Signs|VITAL SIGNS|Vital signs|VITAL signs)\b', elem.text)
            if matches:
                files_with_vital_signs.append(xml_file)
            # vital_signs_pattern.update(matches)
            if matches:
                for match in matches:
                    if match in vital_signs_pattern:
                        vital_signs_pattern[match].append(xml_file)
                    else:
                        vital_signs_pattern[match] = [xml_file]

print(vital_signs_pattern)
## Counter({'Vital Signs': 139, 'Vital signs': 61, 'VITAL SIGNS': 42, 'vital signs': 36})


{'VITAL SIGNS': ['Project3_data/379-03.xml', 'Project3_data/316-02.xml', 'Project3_data/222-03.xml', 'Project3_data/162-03.xml', 'Project3_data/154-03.xml', 'Project3_data/113-02.xml', 'Project3_data/263-02.xml', 'Project3_data/300-02.xml', 'Project3_data/144-01.xml', 'Project3_data/324-03.xml', 'Project3_data/127-01.xml', 'Project3_data/347-02.xml', 'Project3_data/103-01.xml', 'Project3_data/300-01.xml', 'Project3_data/183-05.xml', 'Project3_data/103-02.xml', 'Project3_data/395-02.xml', 'Project3_data/366-01.xml', 'Project3_data/395-01.xml', 'Project3_data/266-02.xml', 'Project3_data/120-04.xml', 'Project3_data/266-03.xml', 'Project3_data/215-02.xml', 'Project3_data/116-01.xml', 'Project3_data/264-01.xml', 'Project3_data/274-04.xml', 'Project3_data/266-04.xml', 'Project3_data/120-03.xml', 'Project3_data/344-02.xml', 'Project3_data/112-01.xml', 'Project3_data/281-03.xml', 'Project3_data/311-01.xml', 'Project3_data/171-03.xml', 'Project3_data/213-04.xml', 'Project3_data/254-01.xml', 'Pr

In [26]:
for key, value in vital_signs_pattern.items():
    count = 0
    if key == 'vital signs':
        for file in value:
            tree = ET.parse(file)
            root = tree.getroot()
            vital_signs = []
            match_found = False
            for elem in root.iter('TEXT'):
                if elem.text:
                    if count < 5:
                        print('file', file)
                        count += 1

file Project3_data/119-01.xml
file Project3_data/107-01.xml
file Project3_data/316-02.xml
file Project3_data/131-01.xml
file Project3_data/107-02.xml


In [17]:
# Define the patterns for the vital signs
vital_signs_patterns = {
    'temperature': r'Temperature (\d+)',
    'pulse': r'Pulse (\d+)',
    'respiratory rate': r'respiratory rate (\d+)',
    'blood pressure': r'(blood pressure|BP) (\d+/\d+)',
    'oxygen saturation': r'Oxygen saturation (\d+%)',
}

frequency = {sign: 0 for sign in vital_signs_patterns.keys()}
no_matches_found = []
for key, value in vital_signs_pattern.items():
    count = 0
    if key == 'VITAL SIGNS':
        for file in value:
            tree = ET.parse(file)
            root = tree.getroot()
            vital_signs = []
            match_found = False
            for elem in root.iter('TEXT'):
                if elem.text:
                    for sign, pattern in vital_signs_patterns.items():
                        matches = re.findall(pattern, elem.text, re.IGNORECASE)
                        if matches:
                            frequency[sign] += 1
                            vital_signs.append({'sign': sign, 'value': matches[0]})
                            match_found = True
            vitalSign_df = pd.DataFrame(vital_signs)
            if not match_found:
                no_matches_found.append(file)
                print(f'No matches found in file: {file}')

print(frequency)


No matches found in file: Project3_data/316-02.xml
No matches found in file: Project3_data/222-03.xml
No matches found in file: Project3_data/144-01.xml
No matches found in file: Project3_data/324-03.xml
No matches found in file: Project3_data/127-01.xml
No matches found in file: Project3_data/103-02.xml
No matches found in file: Project3_data/395-02.xml
No matches found in file: Project3_data/264-01.xml
No matches found in file: Project3_data/112-01.xml
No matches found in file: Project3_data/157-02.xml
No matches found in file: Project3_data/192-03.xml
{'temperature': 15, 'pulse': 20, 'respiratory rate': 11, 'blood pressure': 22, 'oxygen saturation': 4}


In [23]:
len(no_matches_found)

11

In [22]:
for file in no_matches_found:
    tree = ET.parse(file)
    root = tree.getroot()
    for elem in root.iter('TEXT'):
        match = re.search(r'VITAL SIGNS:  (.*?:)', elem.text, re.IGNORECASE | re.DOTALL)
        if match:
            vital_signs_text = match.group(1)
            print('NO MATCH',vital_signs_text)

NO MATCH He is afebrile, vital signs 

are stable.  GENERAL:
NO MATCH Vital signs remained stable, 
including blood pressure.  HEENT:
NO MATCH Normal.  HEENT:
NO MATCH Normal.  HEENT:
NO MATCH Normal, with the 

exception of a low grade temperature of 99.7.  HEENT:
NO MATCH Afebrile.  Normal vital signs. 

Bilaterally equal blood pressures.  GENERAL APPEARANCE:
NO MATCH Her temperature is 98.  Her 

normal temperature is about 96.5.  She is not tachycardic.  

EXTREMITIES:
NO MATCH Stable and normal.  HEENT:
NO MATCH The patient is afebrile. 
Vital signs are normal.  HEENT:
